<a href="https://colab.research.google.com/github/Quanzan1/Projet1/blob/main/Copie_de_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import des libraries

In [3]:
import numpy as np
import torch
import torch.nn as nn
import os
from scipy.io import wavfile

## Téléchargement du dataset

In [2]:
!gdown 1Q3sbordKpx65ExOvqtM4fif7rDXhIXwS
!unzip datasetaudio.zip

Downloading...
From: https://drive.google.com/uc?id=1Q3sbordKpx65ExOvqtM4fif7rDXhIXwS
To: /content/datasetaudio.zip
100% 382M/382M [00:02<00:00, 152MB/s]
Archive:  datasetaudio.zip
replace train/y_train/2168-a.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train/y_train/2168-a.wav  
replace train/y_train/71-a.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace train/y_train/71-a.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y,a
  inflating: train/y_train/71-a.wav  
replace train/y_train/3948-b.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/y_train/1238-b.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y a
  inflating: train/y_train/1238-b.wav  
replace train/y_train/34-a.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace train/y_train/34-a.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y a
  inflating: train/y_train/34-a.wav  
replace train/y_train/2948-b.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Sinon, le lien est ici: https://drive.google.com/file/d/1Q3sbordKpx65ExOvqtM4fif7rDXhIXwS/view?usp=sharing

## Load le dataset

### Load le dataset de train
Pour load un fichier .wav, on utilise la fonction `wavfile.read()` de `scipy.io` qui nous retourne un tuple `(samplerate, data)`. `data` est un array numpy de dimension 1, contenant les valeurs du signal audio. Le nombre d'échantillons par seconde pour nos données est de 4000, donc chaque fichier .wav contient 6000 valeurs car la durée du signal est de 1.5 secondes. `data` est donc de shape `(6000,)`.

Il y a 4096 fichiers .wav dans le dossier `train/x_train` et 2048 fichiers .wav dans le dossier `test/x_test`. Donc on peut créer un array numpy de shape `(4096, 6000)` pour les données d'entraînement et un array numpy de shape `(2048, 6000)` pour les données de test.

In [4]:
X_train = []
Y1_train = []
Y2_train = []

N_TRAIN = 4096

for i in range(N_TRAIN):
    x = wavfile.read(f"train/x_train/{i}.wav")[1]
    y1 = wavfile.read(f"train/y_train/{i}-a.wav")[1]
    y2 = wavfile.read(f"train/y_train/{i}-b.wav")[1]

    X_train.append(x)
    Y1_train.append(y1)
    Y2_train.append(y2)

X_train = np.array(X_train)
Y1_train = np.array(Y1_train)
Y2_train = np.array(Y2_train)

print(X_train.shape)
print(Y1_train.shape)
print(Y2_train.shape)

(4096, 6000)
(4096, 6000)
(4096, 6000)


### Load le dataset de test

In [5]:
X_test = []

N_TEST = 512

for i in range(N_TEST):
    x = wavfile.read(f"test/x_test/{i}.wav")[1]
    X_test.append(x)

X_test = np.array(X_test)

print(X_test.shape)

(512, 6000)


## Lecture des données

In [6]:
import IPython.display as ipd

SAMPLERATE = 4000

In [5]:

ipd.Audio(X_train[0], rate=SAMPLERATE)

In [7]:
ipd.Audio(Y1_train[0], rate=SAMPLERATE)

In [8]:
ipd.Audio(Y2_train[0], rate=SAMPLERATE)

## Batch les données

In [7]:
batch_size = 16
X_train_reshaped = X_train.reshape(-1, batch_size, 6000)
Y1_train_reshaped = Y1_train.reshape(-1, batch_size, 6000)
Y2_train_reshaped = Y2_train.reshape(-1, batch_size, 6000)

print(X_train_reshaped.shape)
print(Y1_train_reshaped.shape)
print(Y2_train_reshaped.shape)

(256, 16, 6000)
(256, 16, 6000)
(256, 16, 6000)


## Convertir les données en torch.tensor

In [7]:
X_train_torch = torch.from_numpy(X_train_reshaped).float()
Y1_train_torch = torch.from_numpy(Y1_train_reshaped).float()
Y2_train_torch = torch.from_numpy(Y2_train_reshaped).float()

## Faire la même chose pour le dataset de test

In [8]:
X_test_reshaped = X_test.reshape(-1, batch_size, 6000)
X_test_torch = torch.from_numpy(X_test_reshaped).float()

print(X_test_torch.shape)

torch.Size([32, 16, 6000])


## Créer le modèle

In [8]:
class Baseline(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(16*6000, 6000*16)
        self.ReLU = nn.ReLU()
        self.conv1 = nn.Conv1d(1, 32, 5)
        self.conv2 = nn.Conv1d(32, 64, 5)

        self.padding = nn.ReplicationPad1d(2)

    def forward(self, x):
        # x: (batch_size, 6000)
        x = x.unsqueezed(1)
        x = self.conv1(x)
        x = self.padding(x)
        x = self.ReLU(x)
        x = self.conv2(x)
        x = self.padding(x)
        x = self.ReLU(x)



        x = nn.Flatten(1,2)


        x1, x2 = x[:, :6000], x[:, 6000:]
        # x1: (batch_size, 6000)
        # x2: (batch_size, 6000)

        return x1, x2

## Boucle d'entraînement

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = nn.MSELoss()
epoque = 4
model = Baseline().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for j in range(epoque):
  for i in range(len(X_train_torch)):
      optimizer.zero_grad()
      X = X_train_torch[i].to(device)
      Y1 = Y1_train_torch[i].to(device)
      Y2 = Y2_train_torch[i].to(device)
      print(Y1.shape)
      break
      Y1_pred, Y2_pred = model(X)
      loss = loss_fn(Y1_pred, Y1) + loss_fn(Y2_pred, Y2)

      loss.backward()
      optimizer.step()
      if i % 10 == 0:
          print(f"Loss {loss.item():.4f}")

## Prédiction

In [ ]:
model.eval()

predictions = np.array([])
predictions = predictions.reshape(0, 2, 6000)

for i in range(len(X_test_torch)):
    X = X_test_torch[i].to(device)
    with torch.no_grad():
        Y1_pred, Y2_pred = model(X)
        Y_pred = torch.stack([Y1_pred, Y2_pred], dim=1)
    predictions = np.concatenate([predictions, Y_pred.cpu().numpy()])

np.save("predictions.npy", predictions)
!zip predictions.zip predictions.npy